In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
# warnings.filterwarnings('ignore')
# warnings.simplefilter('ignore')

import tensorflow as tf

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

import tensorflow_hub as thub
import tensorflow.keras.backend as K

from lib.data_utils import *

In [2]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation
from tensorflow.keras.losses import CategoricalCrossentropy

In [3]:
# from tensorflow.keras.models import Model
from tensorflow.keras.layers import InputLayer, Reshape
from tensorflow.keras import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical


In [4]:
from art.classifiers import KerasClassifier
from art.attacks import FastGradientMethod
from art.attacks.extraction import CopycatCNN, FunctionallyEquivalentExtraction, KnockoffNets
import art

In [5]:
def sample_by_class(data, labels, num_samples=100):
    sample_data = []
    sample_labels = []
    unq_labels = list(range(labels.shape[1]))
    for label in unq_labels:
        idx = labels[:,label]==1
        sample_set = data[idx][0:num_samples].copy()
        label_set = labels[idx][0:num_samples].copy()
        sample_data.append(sample_set)
        sample_labels.append(label_set)
    
    sample_data = np.concatenate(sample_data)
    sample_labels = np.concatenate(sample_labels)
    print(sample_data.shape, sample_labels.shape)
    return sample_data, sample_labels

def subset_data(data, labels, fraction=5):
    data_size = data.shape[0]
    out_size = int(data_size*fraction/100)
    idx = np.random.choice(data_size, out_size, replace=False)
    out_data = data[idx].copy()
    out_labels = labels[idx].copy()
    
    print(out_data.shape, out_labels.shape)
    return out_data, out_labels

In [6]:
# Load the raw CIFAR-10 data
cifar10_dir = 'lib/datasets/cifar-10-batches-py'
x_train, y_train, x_test, y_test = load_cifar10(cifar10_dir)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [7]:
testd = x_train[0:1000].copy()
testl = y_train[0:1000].copy()

In [8]:
def build_ganeval_model(enable_logits=True):
    K.clear_session()
    model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
    ganeval_module = thub.Module(model_url)
    
    model = Sequential()
    model.add(InputLayer(input_shape=(32,32,3)))
    model.add(thub.KerasLayer(ganeval_module))
    if enable_logits:
        model.add(Activation('linear'))
    else: 
        model.add(Activation('softmax'))
        
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                  loss=CategoricalCrossentropy(from_logits=enable_logits),
                  metrics=['accuracy'])
    
    return model

In [9]:
def build_ganeval_model_flat(enable_logits=True):
    K.clear_session()
    model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"
    ganeval_module = thub.Module(model_url)
    
    model = Sequential()
#     model.add(InputLayer(input_shape=(32*32*3,)))
    model.add(Reshape((32,32,1),input_shape=(32*32*1,)))
    model.add(Conv2D(3, (3, 3), activation='relu', padding='same'))
    model.add(thub.KerasLayer(ganeval_module))
    if enable_logits:
        model.add(Activation('linear'))
    else: 
        model.add(Activation('softmax'))
        
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                  loss=CategoricalCrossentropy(from_logits=enable_logits),
                  metrics=['accuracy'])
    
    return model

In [10]:
ge_flat_clf = build_ganeval_model_flat()
ge_flat_clf.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 3)         30        
_________________________________________________________________
keras_layer (KerasLayer)     (None, 10)                7796426   
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 7,796,456
Trainable params: 30
Non-trainable params: 7,796,426
_________________________________________________________________


In [12]:
ge_cifar_clf = build_ganeval_model(enable_logits=True)
ge_cifar_clf.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 10)                7796426   
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


### Testing the retrieved hub model by evaluating on train data
Should give us 100% or something close to that

In [13]:
ge_cifar_clf.evaluate(testd, testl)

1000/1000 [==============================] - 2s 2ms/sample - loss: 0.0017 - acc: 1.0000


[0.001668029203079641, 1.0]

### Testing a sample attack

In [21]:
classifier = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=True)
attack_fgsm = FastGradientMethod(classifier=classifier, eps=0.4)

In [22]:
x_test_adv = attack_fgsm.generate(testd.copy())

In [23]:
ge_cifar_clf.evaluate(x_test_adv, testl)

1000/1000 [==============================] - 2s 2ms/sample - loss: 6.9332 - acc: 0.1020


[6.933242935180664, 0.102]

### Define the substitute classifier for our model

In [13]:
# def build_substitute_model(enable_logits=True):
#     model = tf.keras.Sequential( )
#     model.add( Conv2D( 32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32,32,3) ) )
#     model.add( Conv2D( 64, (3, 3), padding='same', activation='relu' ) )
#     model.add( MaxPooling2D( pool_size=(2, 2) ) )
#     model.add( Flatten( ) )
#     model.add( Dense( 128, activation='relu' ) )
#     if enable_logits:
#         model.add( Dense(10, activation='linear' ) )
#     else:
#         model.add( Dense(10, activation='softmax' ) )
        
#     model.compile(loss=tf.keras.losses.CategoricalCrossentropy(from_logits=enable_logits),
#                   optimizer='adam',
#                   metrics=['accuracy'] )
    

    
#     return model

In [24]:
def build_substitute_model(enable_logits=True):
    model = tf.keras.Sequential( )
    model.add(Conv2D(32, (3, 3), padding="same", input_shape=(32,32,3)))
    model.add(Activation("relu"))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding="same"))
    model.add(Activation("relu"))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation("relu"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation("relu"))
    model.add(Dropout(0.5))
    model.add(Dense(10))
    if enable_logits:
        model.add(Activation("linear"))
    else:
        model.add(Activation("softmax"))
        
    model.compile(loss=CategoricalCrossentropy(from_logits=enable_logits),
                  optimizer='adam',
                  metrics=['accuracy'] )
    
    return model

In [25]:
x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=100)
x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=100)

(1000, 32, 32, 3) (1000, 10)
(1000, 32, 32, 3) (1000, 10)


In [95]:
def extraction_attack(ART_attack, x_train, y_train, fractions, epoch=30, verbose=0):
    
    data_size = x_train.shape[0]
    epochs = epoch
    
    ### Train and Test sample data from each class to get final results
    print("Extract 100 Training samples from each class:")
    x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=100)
    print("Extract 100 Test samples from each class:")
    x_test_adv, y_test_adv = sample_by_class(x_test, y_test, num_samples=100)
    
    ge_cifar_clf = build_ganeval_model()
    cloud_art_clf = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=True)
    
    loss1, acc1 = cloud_art_clf._model.evaluate(x_train_adv, y_train_adv, verbose=verbose)
    loss2, acc2 = cloud_art_clf._model.evaluate(x_test_adv, y_test_adv, verbose=verbose)
    
    clf_results = []
    
    for each_frac in fractions:
        max_queries = int(data_size*each_frac/100)
        print("Attacking the victim with %d percent of training data: %d queries..."%(each_frac, max_queries))
#         partial_x_train, partial_y_train = subset_data(x_train, y_train, fraction=each_frac)
        
        sub_clf = build_substitute_model()
        stolen_clf = KerasClassifier(model=sub_clf, clip_values=(0, 1), use_logits=True)
        
        attack = ART_attack(classifier=cloud_art_clf, 
                        batch_size_fit=32, 
                        batch_size_query=32,
                        nb_epochs=epochs,
                        nb_stolen=max_queries)
        
        stolen_clf = attack.extract(x_train, thieved_classifier=stolen_clf)

        loss3, acc3 = stolen_clf._model.evaluate(x_train_adv, y_train_adv, verbose=verbose)
        loss4, acc4 = stolen_clf._model.evaluate(x_test_adv, y_test_adv, verbose=verbose)
        
        clf_results.append((ART_attack.__name__, each_frac, max_queries, epochs, (round(loss1,4), acc1), (round(loss2,4), acc2), 
                            (round(loss3,4), acc3), (round(loss4,4), acc4)))
        
        results_df = pd.DataFrame(clf_results, columns=['Attack', 'Fraction of data', 'Max queries', 'Epochs trained', 'Victim clf on Train data',
                                                       'Victim clf on Test data', 'Stolen clf on Train data', 
                                                        'Stolen clf on Test data'])
    return results_df
        
        

In [96]:
KON_results_20e = extraction_attack(KnockoffNets, x_train, y_train, [5,10,20,30], epoch=20)
KON_results_30e = extraction_attack(KnockoffNets, x_train, y_train, [5,10,20,30], epoch=30)
KON_results_40e = extraction_attack(KnockoffNets, x_train, y_train, [5,10,20,30], epoch=40)

Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Attacking the victim with 10 percent of training data: 5000 queries...
Attacking the victim with 20 percent of training data: 10000 queries...
Attacking the victim with 30 percent of training data: 15000 queries...
Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Attacking the victim with 10 percent of training data: 5000 queries...
Attacking the victim with 20 percent of training data: 10000 queries...
Attacking the victim with 30 percent of training data: 15000 queries...
Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Attacking the victim with 10 percent of training data: 5000 queries...
Attacking the victim with 20 percent of training data: 10000 queries...
Attacking the victim with 30 percent of training data: 15000 queries...


In [97]:
KON_results_20e.head()

,Attack,Fraction of data,Max queries,Epochs trained,Victim clf on Train data,Victim clf on Test data,Stolen clf on Train data,Stolen clf on Test data
0,KnockoffNets,5,2500,20,"(0.0017, 1.0)","(0.2134, 0.944)","(1.8743, 0.52)","(1.8492, 0.515)"
1,KnockoffNets,10,5000,20,"(0.0017, 1.0)","(0.2134, 0.944)","(1.3711, 0.619)","(1.5513, 0.583)"
2,KnockoffNets,20,10000,20,"(0.0017, 1.0)","(0.2134, 0.944)","(1.0375, 0.723)","(1.2095, 0.653)"
3,KnockoffNets,30,15000,20,"(0.0017, 1.0)","(0.2134, 0.944)","(0.7953, 0.75)","(1.0285, 0.697)"


In [98]:
KON_results_30e.head()

,Attack,Fraction of data,Max queries,Epochs trained,Victim clf on Train data,Victim clf on Test data,Stolen clf on Train data,Stolen clf on Test data
0,KnockoffNets,5,2500,30,"(0.0017, 1.0)","(0.2134, 0.944)","(2.1656, 0.545)","(2.3571, 0.525)"
1,KnockoffNets,10,5000,30,"(0.0017, 1.0)","(0.2134, 0.944)","(1.378, 0.636)","(1.5988, 0.593)"
2,KnockoffNets,20,10000,30,"(0.0017, 1.0)","(0.2134, 0.944)","(1.091, 0.726)","(1.4035, 0.66)"
3,KnockoffNets,30,15000,30,"(0.0017, 1.0)","(0.2134, 0.944)","(0.7677, 0.791)","(0.9552, 0.708)"


In [99]:
KON_results_40e.head()

,Attack,Fraction of data,Max queries,Epochs trained,Victim clf on Train data,Victim clf on Test data,Stolen clf on Train data,Stolen clf on Test data
0,KnockoffNets,5,2500,40,"(0.0017, 1.0)","(0.2134, 0.944)","(2.6595, 0.506)","(2.5057, 0.52)"
1,KnockoffNets,10,5000,40,"(0.0017, 1.0)","(0.2134, 0.944)","(1.5156, 0.65)","(1.7004, 0.603)"
2,KnockoffNets,20,10000,40,"(0.0017, 1.0)","(0.2134, 0.944)","(1.2484, 0.714)","(1.5209, 0.642)"
3,KnockoffNets,30,15000,40,"(0.0017, 1.0)","(0.2134, 0.944)","(0.8399, 0.772)","(1.0338, 0.71)"


In [100]:
CCC_results_20e = extraction_attack(CopycatCNN, x_train, y_train, [5,10,20,30], epoch=20)
CCC_results_30e = extraction_attack(CopycatCNN, x_train, y_train, [5,10,20,30], epoch=30)
CCC_results_40e = extraction_attack(CopycatCNN, x_train, y_train, [5,10,20,30], epoch=40)

Extract 100 Training samples from each class:
(1000, 32, 32, 3) (1000, 10)
Extract 100 Test samples from each class:
(1000, 32, 32, 3) (1000, 10)
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Epoch 1/20
78/78 [==============================] - 3s 41ms/step - loss: 2.1659 - acc: 0.1711
Epoch 2/20
78/78 [==============================] - 2s 29ms/step - loss: 1.9490 - acc: 0.2632
Epoch 3/20
78/78 [==============================] - 2s 29ms/step - loss: 1.7630 - acc: 0.3337
Epoch 4/20
78/78 [==============================] - 2s 29ms/step - loss: 1.6508 - acc: 0.3986
Epoch 5/20
78/78 [==============================] - 2s 29ms/step - loss: 1.5428 - acc: 0.4279
Epoch 6/20
78/78 [==============================] - 2s 29ms/step - loss: 1.4630 - acc: 0.4708
Epoch 7/20
78/78 [==============================] - 2s 29ms/step - loss: 1.4040 - acc: 0.4888
Epoch 8/20
78/78 [==============================] - 2s 29ms/step - loss: 1.3108 - acc: 0.5232
Epoch 9/20
78/78 [==============================] - 2s 29ms/step - loss: 1.1893 - acc: 0.5681
Epoch 10/20
78/78 [==============================] - 2s 29ms/step - loss: 1.1107 - a

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Epoch 1/30
78/78 [==============================] - 3s 42ms/step - loss: 2.2103 - acc: 0.1727
Epoch 2/30
78/78 [==============================] - 2s 29ms/step - loss: 1.9747 - acc: 0.2680
Epoch 3/30
78/78 [==============================] - 2s 29ms/step - loss: 1.7891 - acc: 0.3321
Epoch 4/30
78/78 [==============================] - 2s 29ms/step - loss: 1.7185 - acc: 0.3886
Epoch 5/30
78/78 [==============================] - 2s 29ms/step - loss: 1.5537 - acc: 0.4275
Epoch 6/30
78/78 [==============================] - 2s 29ms/step - loss: 1.4991 - acc: 0.4511
Epoch 7/30
78/78 [==============================] - 2s 29ms/step - loss: 1.3731 - acc: 0.5008
Epoch 8/30
78/78 [==============================] - 2s 29ms/step - loss: 1.3143 - acc: 0.5244
Epoch 9/30
78/78 [==============================] - 2s 29ms/step - loss: 1.2144 - acc: 0.5573
Epoch 10/30
78/78 [==============================] - 2s 29ms/step - loss: 1.1615 - a

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Attacking the victim with 5 percent of training data: 2500 queries...
Epoch 1/40
78/78 [==============================] - 3s 41ms/step - loss: 2.2184 - acc: 0.1759
Epoch 2/40
78/78 [==============================] - 2s 29ms/step - loss: 2.0340 - acc: 0.2516
Epoch 3/40
78/78 [==============================] - 2s 29ms/step - loss: 1.8863 - acc: 0.3169
Epoch 4/40
78/78 [==============================] - 2s 29ms/step - loss: 1.7277 - acc: 0.3602
Epoch 5/40
78/78 [==============================] - 2s 29ms/step - loss: 1.5897 - acc: 0.4323
Epoch 6/40
78/78 [==============================] - 2s 29ms/step - loss: 1.4913 - acc: 0.4555
Epoch 7/40
78/78 [==============================] - 2s 29ms/step - loss: 1.3925 - acc: 0.4940
Epoch 8/40
78/78 [==============================] - 2s 29ms/step - loss: 1.3684 - acc: 0.5164
Epoch 9/40
78/78 [==============================] - 2s 29ms/step - loss: 1.2399 - acc: 0.5537
Epoch 10/40
78/78 [==============================] - 2s 29ms/step - loss: 1.1083 - a

In [101]:
CCC_results_20e.head()

,Attack,Fraction of data,Max queries,Epochs trained,Victim clf on Train data,Victim clf on Test data,Stolen clf on Train data,Stolen clf on Test data
0,CopycatCNN,5,2500,20,"(0.0017, 1.0)","(0.2134, 0.944)","(1.566, 0.539)","(1.6998, 0.532)"
1,CopycatCNN,10,5000,20,"(0.0017, 1.0)","(0.2134, 0.944)","(1.5256, 0.623)","(1.698, 0.584)"
2,CopycatCNN,20,10000,20,"(0.0017, 1.0)","(0.2134, 0.944)","(1.0333, 0.726)","(1.1957, 0.665)"
3,CopycatCNN,30,15000,20,"(0.0017, 1.0)","(0.2134, 0.944)","(0.7759, 0.766)","(0.9894, 0.69)"


In [102]:
CCC_results_30e.head()

,Attack,Fraction of data,Max queries,Epochs trained,Victim clf on Train data,Victim clf on Test data,Stolen clf on Train data,Stolen clf on Test data
0,CopycatCNN,5,2500,30,"(0.0017, 1.0)","(0.2134, 0.944)","(2.0025, 0.535)","(1.9658, 0.524)"
1,CopycatCNN,10,5000,30,"(0.0017, 1.0)","(0.2134, 0.944)","(1.4955, 0.646)","(1.7299, 0.595)"
2,CopycatCNN,20,10000,30,"(0.0017, 1.0)","(0.2134, 0.944)","(1.048, 0.723)","(1.2566, 0.675)"
3,CopycatCNN,30,15000,30,"(0.0017, 1.0)","(0.2134, 0.944)","(0.6689, 0.806)","(1.0855, 0.694)"


In [103]:
CCC_results_40e.head()

,Attack,Fraction of data,Max queries,Epochs trained,Victim clf on Train data,Victim clf on Test data,Stolen clf on Train data,Stolen clf on Test data
0,CopycatCNN,5,2500,40,"(0.0017, 1.0)","(0.2134, 0.944)","(2.3161, 0.557)","(2.2642, 0.522)"
1,CopycatCNN,10,5000,40,"(0.0017, 1.0)","(0.2134, 0.944)","(1.7464, 0.639)","(2.0327, 0.55)"
2,CopycatCNN,20,10000,40,"(0.0017, 1.0)","(0.2134, 0.944)","(1.2221, 0.721)","(1.4793, 0.668)"
3,CopycatCNN,30,15000,40,"(0.0017, 1.0)","(0.2134, 0.944)","(0.7929, 0.788)","(0.9659, 0.725)"


In [106]:
EA_results = pd.concat([KON_results_20e,KON_results_30e,KON_results_40e,CCC_results_20e,CCC_results_30e,CCC_results_40e])
EA_results.head(24)

,Attack,Fraction of data,Max queries,Epochs trained,Victim clf on Train data,Victim clf on Test data,Stolen clf on Train data,Stolen clf on Test data
0,KnockoffNets,5,2500,20,"(0.0017, 1.0)","(0.2134, 0.944)","(1.8743, 0.52)","(1.8492, 0.515)"
1,KnockoffNets,10,5000,20,"(0.0017, 1.0)","(0.2134, 0.944)","(1.3711, 0.619)","(1.5513, 0.583)"
2,KnockoffNets,20,10000,20,"(0.0017, 1.0)","(0.2134, 0.944)","(1.0375, 0.723)","(1.2095, 0.653)"
3,KnockoffNets,30,15000,20,"(0.0017, 1.0)","(0.2134, 0.944)","(0.7953, 0.75)","(1.0285, 0.697)"
0,KnockoffNets,5,2500,30,"(0.0017, 1.0)","(0.2134, 0.944)","(2.1656, 0.545)","(2.3571, 0.525)"
1,KnockoffNets,10,5000,30,"(0.0017, 1.0)","(0.2134, 0.944)","(1.378, 0.636)","(1.5988, 0.593)"
2,KnockoffNets,20,10000,30,"(0.0017, 1.0)","(0.2134, 0.944)","(1.091, 0.726)","(1.4035, 0.66)"
3,KnockoffNets,30,15000,30,"(0.0017, 1.0)","(0.2134, 0.944)","(0.7677, 0.791)","(0.9552, 0.708)"
0,KnockoffNets,5,2500,40,"(0.0017, 1.0)","(0.2134, 0.944)","(2.6595, 0.506)","(2.5057, 0.52)"
1,KnockoffNets,10,5000,40,"(0.0017, 1.0)","(0.2134, 0.944)","(1.5156, 0.65)","(1.7004, 0.603)"


In [105]:
EA_results.to_csv("Extraction attacks results.csv")

In [42]:
partial_x_train, partial_y_train = subset_data(x_train, y_train, fraction=30)

(15000, 32, 32, 3) (15000, 10)


In [43]:
ge_cifar_clf = build_ganeval_model()
sub_clf1 = build_substitute_model()
sub_clf2 = build_substitute_model()
# sub_clf1.summary()
# sub_clf2.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [44]:
cloud_art_clf = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=True)
stolen_clf1 = KerasClassifier(model=sub_clf1, clip_values=(0, 1), use_logits=True)
stolen_clf2 = KerasClassifier(model=sub_clf2, clip_values=(0, 1), use_logits=True)

In [37]:
CCC_results = []

In [28]:
attack_CCC = CopycatCNN(classifier=cloud_art_clf, 
                        batch_size_fit=32, 
                        batch_size_query=32,
                        nb_epochs=2,
                        nb_stolen=10000)

In [29]:
stolen_clf1 = attack_CCC.extract(partial_x_train, thieved_classifier=stolen_clf1)

Epoch 1/2
15/15 [==============================] - 1s 85ms/step - loss: 2.3205 - acc: 0.0812
Epoch 2/2
15/15 [==============================] - 0s 28ms/step - loss: 2.2955 - acc: 0.1104


In [ ]:
%%time
attack_KN = KnockoffNets(classifier=cloud_art_clf, 
                        batch_size_fit=32, 
                        batch_size_query=32,
                        nb_epochs=20,
                        nb_stolen=7000,
                        sampling_strategy='adaptive')
stolen_clf2 = attack_KN.extract(partial_x_train, partial_y_train, thieved_classifier=stolen_clf2)

In [14]:
ge_flat_clf = build_ganeval_model_flat()
ge_flat_clf.summary()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 3)         30        
_________________________________________________________________
keras_layer (KerasLayer)     (None, 10)                7796426   
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 7,796,456
Trainable params: 30
Non-trainable params: 7,796,426
_________________________________________________________________


In [15]:
partial_x_train, partial_y_train = subset_data(x_train, y_train, fraction=0.2)

(100, 32, 32, 3) (100, 10)


In [16]:
partial_x_train = partial_x_train[:,:,:,0]

In [17]:
x_mean = partial_x_train.mean()
x_std = partial_x_train.std()

partial_x_train = (partial_x_train.reshape(-1,32*32*1) - x_mean)/x_std
partial_x_train.shape

(100, 1024)

In [18]:
partial_x_train.mean()

2.559064071760986e-16

In [19]:
ge_flat_clf.evaluate(partial_x_train, partial_y_train)

100/100 [==============================] - 0s 3ms/sample - loss: 6.8181 - acc: 0.1200


[6.818088340759277, 0.12]

In [20]:
flat_art_clf = KerasClassifier(model=ge_flat_clf, clip_values=(0, 1), use_logits=True)

attack_FEN = FunctionallyEquivalentExtraction(classifier=flat_art_clf, num_neurons=10)
stolen_clf_FEN = attack_FEN.extract(partial_x_train)

/home/jupyter/adver_tf1_gcp/lib/python3.5/site-packages/art/attacks/extraction/functionally_equivalent_extraction.py:232: RuntimeWarning: divide by zero encountered in true_divide
  t_hat = t_1 + np.divide(y_2 - y_1 - (t_2 - t_1) * m_2, m_1 - m_2)
/home/jupyter/adver_tf1_gcp/lib/python3.5/site-packages/art/attacks/extraction/functionally_equivalent_extraction.py:233: RuntimeWarning: divide by zero encountered in true_divide
  y_hat = y_1 + m_1 * np.divide(y_2 - y_1 - (t_2 - t_1) * m_2, m_1 - m_2)
/home/jupyter/adver_tf1_gcp/lib/python3.5/site-packages/art/attacks/extraction/functionally_equivalent_extraction.py:247: RuntimeWarning: invalid value encountered in less
  np.sum(np.abs((y_hat - y) / y) < rel_diff_value) > fraction_true * self.num_classes
/home/jupyter/adver_tf1_gcp/lib/python3.5/site-packages/art/attacks/extraction/functionally_equivalent_extraction.py:228: RuntimeWarning: divide by zero encountered in true_divide
  if np.sum(np.abs((m_1 - m_2) / m_1) < rel_diff_slope) > fr

KeyboardInterrupt: 

In [40]:
### Sanity check
# stolen_clf1._model.evaluate(partial_x_train, partial_y_train)

stolen_clf2._model.evaluate(partial_x_train, partial_y_train)

# stolen_clf_FEN._model.evaluate(partial_x_train, partial_y_train)

5000/5000 [==============================] - 1s 230us/sample - loss: 1.9642 - acc: 0.2714


[1.9641792720794677, 0.2714]

In [41]:
cloud_art_clf._model.evaluate(x_train_adv, y_train_adv)
stolen_clf1._model.evaluate(x_train_adv, y_train_adv)
stolen_clf2._model.evaluate(x_train_adv, y_train_adv)
# stolen_clf_FEN._model.evaluate(x_train_adv, y_train_adv)

1000/1000 [==============================] - 0s 220us/sample - loss: 1.9562 - acc: 0.2750


[1.9562057876586914, 0.275]

In [33]:
cloud_art_clf._model.evaluate(x_test_adv, y_test_adv)
stolen_clf1._model.evaluate(x_test_adv, y_test_adv)
stolen_clf2._model.evaluate(x_test_adv, y_test_adv)
# stolen_clf_FEN._model.evaluate(x_test_adv, y_test_adv)

1000/1000 [==============================] - 0s 215us/sample - loss: 7.9670 - acc: 0.2700


[7.966978118896485, 0.27]